In [1]:
import numpy as np
from sympy import *
from math import floor
import time

In [16]:
#Signature is stored as tuple (m,i) where m is the monomial and i is the index
#Poly is stored as a polynomial ie. x**a + x**b +...
class LabeledPolynomial:
    def __init__(self, sign, poly, order = 'lex'):
        self.poly = poly
        self.sign = sign
        self.order = order
    
    def subtract(self,G):
        if signature_cmp(self.sign,G.sign,order = self.order):
            sign = self.sign
        else:
            sign = G.sign
        
        poly = self.poly-G.poly
        return(LabeledPolynomial(sign,poly,order = self.order))
    
    def multiply(self,m):
        sign = (self.sign[0]*m,self.sign[1])
        poly = expand(self.poly*m)
        return(LabeledPolynomial(sign,poly,order = self.order))
    
def monomial_cmp(u,v,order = 'lex'):
    #Return True if u > v
    temp = u+v
    if temp == 0:
        return False
    highest_mon = LT(temp,order = order)
    if highest_mon == u:
        return True
    else:
        return False

    
def signature_cmp(f,g,order = 'lex'):
    #Return True if f > g
    if (f[1] < g[1]):
        return True
    if (f[1] == g[1]):
        if monomial_cmp(f[0],g[0]):
            return True
    return False

def CP(F,G):
    LM1 = LM(F.poly,order = F.order)
    LT1 = LT(F.poly,order = F.order)
    LM2 = LM(G.poly,order = G.order)
    LT2 = LT(G.poly,order = G.order)
    numerator = lcm(LM1,LM2)
    u = simplify(numerator/LT1)
    v = simplify(numerator/LT2)
    return((u,F,v,G))

def spoly(pair):
    lbp1 = pair[1].multiply(pair[0])
    lbp2 = pair[3].multiply(pair[2])
    return(lbp1.subtract(lbp2))

def is_divisible_rewritable(F,B,u):
    index = B.index(F)
    for i,G in enumerate(B):
        uF = F.multiply(u)
        if G.sign[1] > uF.sign[1]:
            if not G.poly == 0: 
                lpp = LM(G.poly,order = G.order)
                if rem(uF.sign[0], lpp) == 0:
                    return True
        if index < i: #If G came after F
            if G.sign[1] == F.sign[1]:
                if sympify(G.sign[0]).is_constant(): 
                    return True
                if not sympify(F.sign[0]).is_constant():
                    lppg = LM(G.sign[0], order = G.order)
                    lppf = LM(F.sign[0], order = F.order)
                    lppf = lppf*u
                    if rem(lppf,lppg) == 0:
                        return True
    return False

def makes_div_rew(pair,R):
    F  = pair[1]
    uF = F.multiply(pair[0])
    if R.sign[1] > uF.sign[1]:
        if R.poly == 0:
            lpp = LM(R.poly,order = R.order)
            if rem(uF.sign[0], lpp) == 0:
                return True
    if R.sign[1] == F.sign[1]:
        if sympify(R.sign[0]).is_constant():
            return True
        if not sympify(F.sign[0]).is_constant():
            lppr = LM(R.sign[0], order = R.order)
            lppf = LM(uF.sign[0], order = F.order)
            if rem(lppf,lppr) == 0:
                return True
    G = pair[3]
    vG = G.multiply(pair[2])
    if R.sign[1] > vG.sign[1]:
        lpp = LM(R.poly, order = R.order)
        if rem(vG.sign[0], lpp) == 0:
            return True
        if R.sign[1] == G.sign[1]:
            if sympify(R.sign[0]).is_constant():
                return True
            if not sympify(G.sign[0]).is_constant():
                lppr = LM(R.sign[0], order = R.order)
                lppg = LM(vG.sign[0], order = G.order)
                if rem(lppg, lppr) == 0:
                    return True
    return False

def f5_reduce(F,B):
    can_reduce = True
    while can_reduce:
        if (F.poly == 0):
            return(F)
        can_reduce = False
        for G in B:
            if(G.poly != 0):
                if rem(LM(G.poly,order = G.order),LM(F.poly,order = F.order)) == 0:
                    v = LT(F.poly,order = F.order)/LT(G.poly,order = G.order)
                    vG = G.multiply(v)
                    if signature_cmp(F.sign,vG.sign):
                        if not is_divisible_rewritable(G,B,v):
                                F = F.subtract(vG)
                                can_reduce = True
                                break
                            
    return(F)

In [3]:
def reduce_groebner(G,order = 'lex'):
    H = []
    for i,g in enumerate(G):
        temp = [f for f in G if not f == g]
        m = LM(g,order = order)
        q,r = reduced(m,temp)
        if not r == 0:
            H.append(g/LC(g,order = order))
            
    for i,h in enumerate(H):
        temp = [f for f in H if not f == h]
        q,r = reduced(h,temp)
        H[i] = r
    return(H)

In [14]:
def iterative_f5(G,f,order = 'lex'):
    B = []
    q,r = reduced(f,G)
    if r == 0:
        return(G)
    f = r
    for i,p in enumerate(G+[f]):
        b = LabeledPolynomial((1,i),p,order = order)
        B.append(b)
    n = len(B)
    pairs = []
    for i in range(n-1):
        pairs.append(CP(B[i],B[-1]))
    while len(pairs) > 0:
        cp  = pairs.pop()
        u = cp[0]
        F = cp[1]
        v = cp[2]
        G = cp[3]
        if not is_divisible_rewritable(F,B,u):
            if not is_divisible_rewritable(G,B,v):
                S = spoly(cp)
                P = f5_reduce(S,B)
                B.append(P)
                if P.poly != 0:
                    remove_indices = []
                    for i,cp in enumerate(pairs):
                        if makes_div_rew(cp,P):
                            remove_indices.append(i)
                    pairs = [pairs[i] for i in range(len(pairs)) 
                             if not i in remove_indices]
                    for b in B[:-1]:
                        if(b.poly != 0):
                            cp = CP(b,P)
                            u = cp[0]
                            F = cp[1]
                            v = cp[2]
                            G = cp[3]
                            if not is_divisible_rewritable(F,B,u):
                                if not is_divisible_rewritable(G,B,v):
                                    pairs.append(CP(b,P))
    G = [b.poly for b in B if b.poly != 0]
    G= reduce_groebner(G,order = order)
    return(G)

In [5]:
#Useful functions for both Hypercubes and general Hamming graphs
def read_sets(filename):
    #Read in the sets of a file
    results = []
    with open(filename,'r') as f:
        for line in f.read().splitlines():
            results.append(line.split(','))
    return(results)

def make_linearEqns(A,z):
    #Takes matrix A and variable list z and converts it into a list of linear equations
    z = Matrix(z)
    A = Matrix(A)
    A = A.rref()[0]
    lin_fcns = A*z
    return(list(lin_fcns))

In [6]:
def OneHot(R,k,a,alphabet = None):
    #Converts list of strings to list of one-hot encodings
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    encodings = []
    for r in R:
        encoding = np.zeros((a,k))
        for i in range(k):
            for j,letter in enumerate(alphabet):
                if r[i] == letter:
                    encoding[j,i] = 1
        encodings.append(encoding)
    return(encodings)

def create_polys_improved(k,a):
    #Setup polynomial system for H_k,a without knowing what R is
    variable_string = 'z1'
    for i in range(1,k*a):
        variable_string = variable_string +',z{} '.format(i+1)
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        Pi = []
        func = 0
        #func2 = 0
        for j in range(0,a):
            Pi.append(z[i*a+j]*(z[i*a+j]-1)*(z[i*a+j]+1)) #1st condition zi*(zi-1)*(zi+1)
            func= func + z[i*a+j]**2
            #func2 = func2 + z[i*a+j]
            f = f + z[i*a+j]**2 #f so that sum(zi**2) neq 0
        func = (2-func)*(func)
        Pi.append(func)
        #Pi.append(func2)
        P.append(Pi)
    
    fs = [f-2*i for i in range(1,k+1)]
    return(P,fs,z)

def preprocess_groebner(P,z,k,a):
    G0 = groebner(P[0],order = 'lex')
    G = [g for g in list(G0)]
    for i in range(1,k):
        replacements = [(z[j],z[a*i+j]) for j in range(a)]
        for g in list(G0):
            G.append(g.subs(replacements))
    return(G)

def make_matrix(R,k,a):
    #Converts list of one-hot encodings to the linear system
    temp = [r.flatten('F') for r in R]
    for i in range(k):
        added_row = np.zeros(k*a)
        for j in range(a):
            #This is the 2nd condition where sum(zi)_i*a+1^i*a+a = 0
            added_row[i*a+j] = 1
        temp.append(list(added_row))
    return(np.array(temp,dtype = int))

def check_resolving_improved(R,k,a,alphabet = None):
    (P,fs,z) = create_polys_improved(k,a)
    OH_encodedR = OneHot(R,k,a)
    A = make_matrix(OH_encodedR,k,a)
    lin_fcns = make_linearEqns(A,z)
    pre_G = preprocess_groebner(P,z,k,a)
    G = pre_G
    for f in lin_fcns:
        G = iterative_f5(G,f,order = 'lex')
        G = reduce_groebner(G,order = 'lex')
    for fi in fs:
        Gi = iterative_f5(G,fi,order = 'lex')
        Gi = reduce_groebner(Gi,order = 'lex')
        if not (Gi == [1]):
            return False
    return True

In [7]:
#Using IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_3_5.txt')
R = sets[0]
a = 3
k = 5

In [8]:
(P,fs,z) = create_polys_improved(k,a)

In [9]:
pre_G = preprocess_groebner(P,z,k,a)

In [10]:
OH_encodedR = OneHot(R,k,a)
A = make_matrix(OH_encodedR,k,a)
lin_fcns = make_linearEqns(A,z)

In [17]:
f = lin_fcns[0]
G = iterative_f5(pre_G,f,order = 'lex')

KeyboardInterrupt: 